In [1]:
path_data = '/data'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
#display(data_unfiltered)

In [4]:
len(data_unfiltered)

38479

### Remove subjective topics

In [5]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [6]:
len(subjective_topics)

19

In [7]:
unique_topics = data_unfiltered['topics'].unique()

In [8]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [9]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [10]:
len(data)

36754

### Split topics

In [11]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
#print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
270


In [12]:
len(separate_topics)

45692

In [13]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [14]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [15]:
topics = dict_topic2img.keys()

### Get ground truth

In [16]:
temp = "This is a photo of "
English_topics = [temp + topic for topic in topics]

In [17]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [18]:
#topic2english

In [19]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


In [20]:
len(english_dict_topic2imgGT['This is a photo of spices'])

419

In [30]:
type(lang_list_dict_topic2FNimg)

dict

## Plotting results

In [28]:
import pickle
with open("data/english_dict_topic2FNimg.pkl", 'rb') as f:
    lang_list_dict_topic2FNimg = pickle.load(f)

In [34]:
lang_list_dict_topic2FNimg

{'This is a photo of family snapshots': ['assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a0f3f359814.jpg',
  'assets/5d4bf330cf0b3a0f3f359a20/5d4bf330cf0b3a0f3f359a20.jpg',
  'assets/5d4bea92cf0b3a0f3f34aae6/5d4bea92cf0b3a0f3f34aae6.jpg',
  'assets/5d4bec3bcf0b3a0f3f34d91a/5d4bec3bcf0b3a0f3f34d91a.jpg',
  'assets/5d4bf340cf0b3a0f3f359bf2/5d4bf340cf0b3a0f3f359bf2.jpg',
  'assets/5d4bea70cf0b3a0f3f34a6d0/5d4bea70cf0b3a0f3f34a6d0.jpg',
  'assets/5d4beedbcf0b3a0f3f3522ae/5d4beedbcf0b3a0f3f3522ae.jpg',
  'assets/5d4bee16cf0b3a0f3f350d0a/5d4bee16cf0b3a0f3f350d0a.jpg',
  'assets/5d4be754cf0b3a0f3f3453f8/5d4be754cf0b3a0f3f3453f8.jpg',
  'assets/5d4beff9cf0b3a0f3f35428a/5d4beff9cf0b3a0f3f35428a.jpg',
  'assets/5d4bee35cf0b3a0f3f350f94/5d4bee35cf0b3a0f3f350f94.jpg',
  'assets/5d4beaffcf0b3a0f3f34b7bc/5d4beaffcf0b3a0f3f34b7bc.jpg',
  'assets/5d4be629cf0b3a0f3f3432b3/5d4be629cf0b3a0f3f3432b3.jpg',
  'assets/5d4bef28cf0b3a0f3f352bb2/5d4bef28cf0b3a0f3f352bb2.jpg',
  'assets/5d4bea0ccf0b3a0f3f349b42/5d

In [29]:
with open("data/english_dict_topic2F1.pkl", 'rb') as f:
    lang_list_dict_topic2F1 = pickle.load(f)

In [31]:
labels=["poor", "lower-middle", "upper-middle", "rich"]

#list_of_topic2prompt_dict[1]
data["quartile"] = pd.qcut(data["income"], q=[0, 0.25, 0.5, 0.75, 1],    
                         labels=labels)

In [32]:
# map each image to corresponding topics, countries, income level ... TODO
dict_img2topic, dict_img2country, dict_img2incomelevel, dict_country2region = {}, {}, {}, {}
for list_topics, image_path, country, incomelevel, region in zip(data['topics'], data['imageRelPath'], data['country.name'], data['quartile'], data['region.id']):
    dict_img2topic[image_path] = list_topics
    dict_img2country[image_path] = country
    dict_img2incomelevel[image_path] = incomelevel
    dict_country2region[country] = region

print(len(dict_img2topic))
# print(dict_img2country)

36754


## Analysis of results

In [79]:
from collections import Counter

all_topic_all_income_percountrydf = pd.DataFrame()
all_topic_all_country_perincomedf = pd.DataFrame()
language_recall_df = pd.DataFrame()


    
#### forgotten images block (countrywise)

list_all_countriesFN = []
list_all_incomeFN = []
dict_topic2FNCountry = {}
dict_all_Countries_FN = {}
lang_dict = lang_list_dict_topic2FNimg

for topic in lang_dict:

    #dict_lang2translation = next(item for item in list_dict_lang2translation if topic.split(';')[1].strip() in list(item.values())[0])
    dominant_lang = 'English'
    list_imagesFN = lang_dict[topic]
    list_countriesFN = [dict_img2country[img] for img in list_imagesFN]
    list_incomepercountry = [dict_img2incomelevel[img] for img in list_imagesFN]

    for country in list_countriesFN:
        list_all_countriesFN.append(country)
    for income in list_incomepercountry:
        list_all_incomeFN.append(income)

    dict_topic2FNCountry[topic] = Counter(list_countriesFN)

dict_all_Countries_FN = Counter(list_all_countriesFN)


#### ground truth images block (countrywise)

list_all_countriesGT = []
list_all_incomeGT = []
dict_topic2GTCountry = {}
dict_all_Countries_GT = {}

dict_lang_topic2img = english_dict_topic2imgGT

for topic in dict_lang_topic2img:
    list_imagesGT = dict_lang_topic2img[topic]
    list_countriesGT = [dict_img2country[img] for img in list_imagesGT]
    list_incomepercountryGT = [dict_img2incomelevel[img] for img in list_imagesGT]
    for country in list_countriesGT:
        list_all_countriesGT.append(country)
    for income in list_incomepercountryGT:
        list_all_incomeGT.append(income)

    dict_topic2GTCountry[topic] = Counter(list_countriesGT)


dict_all_Countries_GT = Counter(list_all_countriesGT)



### forgotten images block (income level wise)

list_all_incomelevelsFN = []
dict_topic2FNIncomelevels = {}
dict_all_incomelevels_FN = {}

mapped_dict_lang_topic2FNimg = lang_list_dict_topic2FNimg

for topic in mapped_dict_lang_topic2FNimg:
    list_imagesFN = mapped_dict_lang_topic2FNimg[topic]
    list_incomelevelsFN = [dict_img2incomelevel[img] for img in list_imagesFN]
    for incomelevel in list_incomelevelsFN:
        list_all_incomelevelsFN.append(incomelevel)

    dict_topic2FNIncomelevels[topic] = Counter(list_incomelevelsFN)


dict_all_incomelevels_FN = Counter(list_all_incomelevelsFN)



### ground truth image block (income level wise)

### Income level ground truth

list_all_incomelevelsGT = []
dict_topic2GTIncomelevels = {}
dict_all_incomelevels_GT = {}

mapped_dict_lang_topic2img = english_dict_topic2imgGT
for topic in mapped_dict_lang_topic2img:
    list_imagesGT = mapped_dict_lang_topic2img[topic]
    list_incomelevelsGT = [dict_img2incomelevel[img] for img in list_imagesGT]
    for incomelevel in list_incomelevelsGT: 
        list_all_incomelevelsGT.append(incomelevel)

    dict_topic2GTIncomelevels[topic] = Counter(list_incomelevelsGT)


dict_all_incomelevels_GT = Counter(list_all_incomelevelsGT)


#### dataframe building

################### all_topic_all_income_percountrydf ###########

Forget_ratio_allincome_percountry = Counter({key : dict_all_Countries_FN[key] / dict_all_Countries_GT[key] for key in dict_all_Countries_FN})
Forget_ratio_allincome_percountry_df = pd.DataFrame(Forget_ratio_allincome_percountry.items(),
                    columns = ['Country', 'Forget ratio'])
Forget_ratio_allincome_percountry_df['dominant language'] = [dominant_lang for i in list(Forget_ratio_allincome_percountry_df['Forget ratio'])]
Forget_ratio_allincome_percountry_df['Total images per country'] = Forget_ratio_allincome_percountry_df['Country'].map(dict_all_Countries_GT)
Forget_ratio_allincome_percountry_df['Forgotten images'] = Forget_ratio_allincome_percountry_df['Country'].map(dict_all_Countries_FN)
Forget_ratio_allincome_percountry_df['Recognized images'] = Forget_ratio_allincome_percountry_df['Total images per country'] - Forget_ratio_allincome_percountry_df['Forgotten images']
Forget_ratio_allincome_percountry_df['Recall'] = Forget_ratio_allincome_percountry_df['Recognized images'] / Forget_ratio_allincome_percountry_df['Total images per country']

all_topic_all_income_percountrydf = pd.concat([all_topic_all_income_percountrydf, Forget_ratio_allincome_percountry_df])


####################### alltopic_all_country_perincome ###################

Forget_ratio_all_country_perincome = Counter({key : dict_all_incomelevels_FN[key] / dict_all_incomelevels_GT[key] for key in dict_all_incomelevels_FN})
Forget_ratio_all_country_perincome_df = pd.DataFrame(Forget_ratio_all_country_perincome.items(), columns = ['Income level', 'Forget ratio',])
Forget_ratio_all_country_perincome_df['Total images per income level'] = Forget_ratio_all_country_perincome_df['Income level'].map(dict_all_incomelevels_GT)
Forget_ratio_all_country_perincome_df['Forgotten images'] = Forget_ratio_all_country_perincome_df['Income level'].map(dict_all_incomelevels_FN)
#Forget_rationperincome_allcountry_df = Forget_ratio_percountry_allincome_df
Forget_ratio_all_country_perincome_df['dominant language'] = [dominant_lang for i in list(Forget_ratio_all_country_perincome_df['Forget ratio'])]
Forget_ratio_all_country_perincome_df['Recognized images'] = Forget_ratio_all_country_perincome_df['Total images per income level'] - Forget_ratio_all_country_perincome_df['Forgotten images']
Forget_ratio_all_country_perincome_df['Recall'] = Forget_ratio_all_country_perincome_df['Recognized images'] / Forget_ratio_all_country_perincome_df['Total images per income level']

all_topic_all_country_perincomedf = pd.concat([all_topic_all_country_perincomedf, Forget_ratio_all_country_perincome_df])



######################## all topic, per country, per income #####################

Lang_FN_df = pd.DataFrame()
Lang_FN_df['countries'] = list_all_countriesFN
Lang_FN_df['language'] = [dominant_lang for i in list_all_countriesFN]
Lang_FN_df['income level'] = list_all_incomeFN
Lang_forgotten = Lang_FN_df.groupby(['countries', 'income level']).count()
Lang_forgotten.reset_index()

lang_countries = [x[0] for x in Lang_forgotten.index]
lang_incomelevel = [x[1] for x in Lang_forgotten.index]

Lang_GT_df = pd.DataFrame()
Lang_GT_df['countries'] = list_all_countriesGT
Lang_GT_df['language'] = [dominant_lang for i in list_all_countriesGT]
Lang_GT_df['income level'] = list_all_incomeGT


Lang_GT = Lang_GT_df.groupby(['countries', 'income level']).count()
Lang_GT.reset_index()

Lang_recall_df = pd.DataFrame()

Lang_recall_df['language'] = [dominant_lang for i in lang_countries]
Lang_recall_df['countries'] = lang_countries
Lang_recall_df['incomelevel'] = lang_incomelevel
Lang_recall_df['forgotten'] = list(Lang_forgotten['language'])
Lang_recall_df['ground truth'] =  list(Lang_GT['language'])
Lang_recall_df['recall'] = 1 - (Lang_recall_df['forgotten']/ Lang_recall_df['ground truth'])

language_recall_df = pd.concat([language_recall_df, Lang_recall_df])

    

all_topic_all_income_percountrydf.to_csv('english_all_topic_all_income_percountrydf')
all_topic_all_country_perincomedf.to_csv('english_all_topic_all_country_perincomedf')
language_recall_df.to_csv('english_recall_df')

In [11]:
import pandas as pd
all_topic_all_income_percountrydf = pd.read_csv('english_all_topic_all_income_percountrydf')
all_topic_all_country_perincomedf = pd.read_csv('english_all_topic_all_country_perincomedf')
language_recall_df = pd.read_csv('english_recall_df')

In [12]:
all_topic_all_country_perincomedf

,Unnamed: 0,Income level,Forget ratio,Total images per income level,Forgotten images,dominant language,Recognized images,Recall
0,0,poor,0.776149,11119,8630,English,2489,0.223851
1,1,lower-middle,0.640902,10997,7048,English,3949,0.359098
2,2,upper-middle,0.543945,11446,6226,English,5220,0.456055
3,3,rich,0.564045,12054,6799,English,5255,0.435955


In [3]:
language_recall_df[language_recall_df['countries'] == 'Vietnam']

,Unnamed: 0,language,countries,incomelevel,forgotten,ground truth,recall
125,125,English,Vietnam,lower-middle,331,491,0.325866
126,126,English,Vietnam,rich,77,145,0.468966


In [4]:
## lower income results only (excluding images from rich and up-mid households)

low_lvl_language_recall_df = pd.DataFrame()

unique_countries = list(all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    all_lvl = language_recall_df[language_recall_df['countries'] == country]
    unique_lvls = list(all_lvl['incomelevel'].unique())
    lvls = []
    for lvl in unique_lvls:
        if lvl in ['poor', 'lower-middle']:
            lvls.append(lvl)

    if len(lvls) == 2:

        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == 'poor']
        lower_lvl_low_mid = all_lvl[all_lvl['incomelevel'] == 'lower-middle']

        forgotten = list(lower_lvl_poor['forgotten']) + (lower_lvl_low_mid['forgotten'])
        ground_truth = list(lower_lvl_poor['ground truth']) + (lower_lvl_low_mid['ground truth'])
        recall = (ground_truth - forgotten) / ground_truth
        df = lower_lvl_poor.drop(['forgotten', 'incomelevel', 'recall', 'ground truth'], axis=1)
        df['incomelevel'] = ['poor, lower-middle' for i in df['countries']]
        df['forgotten'] = list(forgotten)
        df['ground truth'] = list(ground_truth)
        df['recall'] = list(recall)
        low_lvl_language_recall_df = pd.concat([low_lvl_language_recall_df, df])

    elif len(lvls) == 1:
        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == lvls[0]]
        df = lower_lvl_poor
        low_lvl_language_recall_df = pd.concat([low_lvl_language_recall_df, df])

    elif len(lvls) == 0:
        df = pd.DataFrame()

low_lvl_language_recall_df.to_csv('english_low_lvl_recall_df') 

In [12]:
import pandas as pd
low_lvl_language_recall_df = pd.read_csv('english_low_lvl_recall_df') 

In [13]:
low_lvl_language_recall_df

,Unnamed: 0.1,Unnamed: 0,language,countries,incomelevel,forgotten,ground truth,recall
0,2,2,English,Bangladesh,poor,555,753,0.262948
1,4,4,English,Bolivia,"poor, lower-middle",224,340,0.341176
2,5,5,English,Brazil,lower-middle,514,778,0.339332
3,8,8,English,Burkina Faso,poor,505,589,0.142615
4,9,9,English,Burundi,poor,280,331,0.154079
5,11,11,English,Cambodia,"poor, lower-middle",450,742,0.393531
6,14,14,English,Cameroon,"poor, lower-middle",358,516,0.306202
7,18,18,English,China,"poor, lower-middle",427,613,0.303426
8,22,22,English,Colombia,"poor, lower-middle",451,660,0.316667
9,25,25,English,Cote d'Ivoire,poor,278,357,0.221289


In [26]:
# higher income images (excluding images from poor and low-mid households)
high_lvl_language_recall_df = pd.DataFrame()

unique_countries = list(all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    all_lvl = language_recall_df[language_recall_df['countries'] == country]
    unique_lvls = list(all_lvl['incomelevel'].unique())
    lvls = []
    for lvl in unique_lvls:
        if lvl in ['rich', 'upper-middle']:
            lvls.append(lvl)

    if len(lvls) == 2:

        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == 'rich']
        lower_lvl_low_mid = all_lvl[all_lvl['incomelevel'] == 'upper-middle']

        forgotten = list(lower_lvl_poor['forgotten']) + (lower_lvl_low_mid['forgotten'])
        ground_truth = list(lower_lvl_poor['ground truth']) + (lower_lvl_low_mid['ground truth'])
        recall = (ground_truth - forgotten) / ground_truth
        df = lower_lvl_poor.drop(['forgotten', 'incomelevel', 'recall', 'ground truth'], axis=1)
        df['incomelevel'] = ['rich, upper-middle' for i in df['countries']]
        df['forgotten'] = list(forgotten)
        df['ground truth'] = list(ground_truth)
        df['recall'] = list(recall)
        high_lvl_language_recall_df = pd.concat([high_lvl_language_recall_df, df])

    elif len(lvls) == 1:
        lower_lvl_poor = all_lvl[all_lvl['incomelevel'] == lvls[0]]
        df = lower_lvl_poor
        high_lvl_language_recall_df = pd.concat([high_lvl_language_recall_df, df])

    elif len(lvls) == 0:
        df = pd.DataFrame()



In [27]:
high_lvl_language_recall_df

,Unnamed: 0,language,countries,incomelevel,forgotten,ground truth,recall
0,0,English,Austria,"rich, upper-middle",292,428,0.317757
6,6,English,Brazil,"rich, upper-middle",695,1182,0.412014
12,12,English,Cambodia,upper-middle,70,146,0.520548
15,15,English,Cameroon,upper-middle,78,165,0.527273
16,16,English,Canada,rich,14,48,0.708333
19,19,English,China,"rich, upper-middle",935,1482,0.369096
23,23,English,Colombia,"rich, upper-middle",1371,2340,0.414103
26,26,English,Czech Republic,rich,236,393,0.399491
27,27,English,Denmark,rich,224,312,0.282051
28,28,English,Egypt,upper-middle,126,284,0.556338


## by continent

In [15]:
African_countries_low = ['Burkina Faso', 'Burundi', 'Ethiopia', 'Liberia', 'Malawi', 'Rwanda', 'Somalia', 'Togo', 'Cameroon', "Cote d'Ivoire", 'Ghana', 'Kenya', 'Nigeria', 'Tanzania', 'Tunisia', 'Zimbabwe', 'South Africa']
American_countries_low = ['Bolivia', 'Haiti', 'Brazil', 'Colombia', 'Guatemala', 'Peru', 'United States']
Asian_countries_low = ['Bangladesh', 'Cambodia', 'India', 'Jordan', 'Mongolia', 'Myanmar', 'Nepal', 'Pakistan', 'Palestine', 'Papua New Guinea', 'Philippines', 'Vietnam', 'China', 'Indonesia', 'Thailand', 'South Korea']
European_countries_low = ['Ukraine', 'Serbia']

In [29]:
African_countries_high = ['Ethiopia', 'Cameroon', 'Egypt', 'Kenya', 'Nigeria', 'Tanzania', 'South Africa']
American_countries_high = ['Brazil', 'Colombia', 'Mexico', 'Canada', 'United States']
Asian_countries_high = ['Cambodia', 'India', 'Iran', 'Jordan', 'Kyrgyzstan','Lebanon', 'Nepal', 'Pakistan', 'Philippines', 'Sri Lanka', 'Vietnam', 'China', 'Kazakhstan','Indonesia', 'Thailand', 'South Korea']
European_countries_high= ['Ukraine', 'Russia', 'Serbia', 'Turkey', 'Austria', 'Czech Republic', 'Denmark', 'France', 'Italy', 'Netherlands', 'Romania', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom']

len(African_countries_high)+ len(American_countries_high) +len(Asian_countries_high)+len(European_countries_high)

43

In [37]:
recall_dict = {}
for i in European_countries_high:
    recall = high_lvl_language_recall_df[high_lvl_language_recall_df['countries']== i]['recall']
    
    recall_dict[i] = round(recall.item(), 3)
    

In [38]:
for i in recall_dict.values():
    print(i)

0.462
0.458
0.495
0.364
0.318
0.399
0.282
0.453
0.43
0.43
0.475
0.423
0.304
0.336
0.528


In [7]:
all_levels = language_recall_df[language_recall_df['countries'] == 'Colombia']
incm_lst = ['poor', 'lower-middle', 'upper-middle', 'rich']
recallforincm = []
for level in incm_lst:
    recall_val = all_levels[all_levels['incomelevel'] == level]['recall'].item()
    recallforincm.append(recall_val)

# poor = all_levels[all_levels['incomelevel'] == 'poor']['recall'].item()
# poor

In [8]:
all_topic_all_income_percountrydf[all_topic_all_income_percountrydf['Country'] == 'Colombia']

,Unnamed: 0,Country,Forget ratio,dominant language,Total images per country,Forgotten images,Recognized images,Recall
22,22,Colombia,0.607333,English,3000,1822,1178,0.392667


## Query for table filling

In [14]:
selected_countries = ['Austria', 'Bolivia', 'Brazil', 'Burkina Faso', 'Burundi', 'Cameroon', 'Canada', 'China', 'Colombia', "Cote d'Ivoire", 'Czech Republic', 'Denmark', 'Egypt', 'France', 'Guatemala', 'Haiti', 'India', 'Indonesia', 'Iran', 'Italy', 'Jordan', 'Kazakhstan', 'Lebanon', 'Mexico', 'Mongolia', 'Myanmar', 'Netherlands', 'Pakistan', 'Palestine', 'Peru', 'Romania', 'Russia', 'Serbia', 'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Togo', 'Tunisia', 'Turkey', 'Ukraine', 'Vietnam']

In [15]:
df_countries = list(low_lvl_language_recall_df['countries'].unique())
df_countries.sort()

lang_countries = [country for country in selected_countries if country in df_countries]
len(lang_countries)

25

In [16]:
## Query for English prompt recall for each country column
English_prompt_recall_list = []

unique_countries = list(all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    recall = all_topic_all_income_percountrydf[all_topic_all_income_percountrydf['Country'] == country]['Recall'].item()
    recall = round(recall, 3)
    English_prompt_recall_list.append(recall)

In [23]:
low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]

,Unnamed: 0.1,Unnamed: 0,language,countries,incomelevel,forgotten,ground truth,recall
41,127,127,English,Zimbabwe,poor,385,450,0.144444


In [17]:
for i in English_prompt_recall_list:
    print(i)

0.318
0.263
0.341
0.383
0.143
0.154
0.414
0.36
0.708
0.35
0.393
0.221
0.399
0.282
0.556
0.368
0.453
0.367
0.374
0.201
0.383
0.398
0.445
0.43
0.457
0.604
0.332
0.659
0.506
0.268
0.141
0.529
0.32
0.258
0.327
0.43
0.331
0.408
0.414
0.172
0.255
0.353
0.475
0.458
0.296
0.426
0.093
0.29
0.476
0.423
0.644
0.304
0.336
0.328
0.375
0.262
0.301
0.364
0.456
0.528
0.53
0.358
0.144


In [18]:
## Query for English prompt recall for each country column for lower-income

English_prompt_recall_list = []

unique_countries = list(all_topic_all_income_percountrydf['Country'].unique())
unique_countries.sort()

for country in unique_countries:
    if country in df_countries:
        recall = low_lvl_language_recall_df[low_lvl_language_recall_df['countries'] == country]['recall'].item()
        recall = round(recall, 3)
        English_prompt_recall_list.append(recall)
    else:
        English_prompt_recall_list.append('-')

In [19]:
for i in English_prompt_recall_list:
    print(i)

-
0.263
0.341
0.339
0.143
0.154
0.394
0.306
-
0.303
0.317
0.221
-
-
-
0.313
-
0.367
0.374
0.201
0.283
0.379
-
-
0.464
-
0.287
-
-
0.268
0.141
-
0.32
0.258
0.295
-
0.261
0.3
0.414
0.172
0.255
0.328
-
-
0.296
0.364
0.093
0.222
0.45
-
-
-
-
0.276
0.312
0.262
0.301
-
0.433
-
0.463
0.326
0.144


In [31]:
# for i in df_countries:
#     print(i)

## Correlation calc

In [26]:
import numpy as np
from scipy.stats import pearsonr

In [22]:
Lower_income_eng_prompt_nllb = [i for i in English_prompt_recall_list if i != '-']

In [23]:
len(Lower_income_eng_prompt_nllb)

42

In [35]:
# for i in Lower_income_eng_prompt_mclip:
#     print(i)

In [25]:
Lower_income_eng_prompt_mclip = np.load('mclip_low_engprompt_recall.npy')

In [27]:
corr, _ = pearsonr(Lower_income_eng_prompt_mclip, Lower_income_eng_prompt_nllb)
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.875


In [26]:
# Query for filling out the Income levels present
incomelevel_list = []
country_income_level_dict = {}

for country in unique_countries:
    income_levels = list(language_recall_df[language_recall_df['countries'] == country]['incomelevel'])
    incomelevel_list.append(income_levels)
    country_income_level_dict[country] = income_levels

In [27]:
#country_income_level_dict

In [28]:
## Query for English prompt recall per income group for each country column


#recall_incomelevel_list = []
country_income_level_recall_dict = {}

for country in unique_countries:
    all_levels = language_recall_df[language_recall_df['countries'] == country]
    incm_lst = ['poor', 'lower-middle', 'upper-middle', 'rich']
    recallforincm = []
    for level in incm_lst:
        if level in list(language_recall_df[language_recall_df['countries'] == country]['incomelevel']):
            recall_val = round(all_levels[all_levels['incomelevel'] == level]['recall'].item(), 3)
            recallforincm.append(recall_val)
        else: 
            recall_val = '-'
            recallforincm.append(recall_val)
    country_income_level_recall_dict[country] = recallforincm

country_income_level_recall_dict

{'Austria': ['-', '-', 0.389, 0.282],
 'Bangladesh': [0.263, '-', '-', '-'],
 'Bolivia': [0.358, 0.335, '-', '-'],
 'Brazil': ['-', 0.339, 0.44, 0.351],
 'Burkina Faso': [0.143, '-', '-', '-'],
 'Burundi': [0.154, '-', '-', '-'],
 'Cambodia': [0.373, 0.405, 0.521, '-'],
 'Cameroon': [0.164, 0.377, 0.527, '-'],
 'Canada': ['-', '-', '-', 0.708],
 'China': [0.197, 0.317, 0.4, 0.328],
 'Colombia': [0.272, 0.363, 0.412, 0.419],
 "Cote d'Ivoire": [0.221, '-', '-', '-'],
 'Czech Republic': ['-', '-', '-', 0.399],
 'Denmark': ['-', '-', '-', 0.282],
 'Egypt': ['-', '-', 0.556, '-'],
 'Ethiopia': ['-', 0.313, 0.345, 0.492],
 'France': ['-', '-', 0.49, 0.429],
 'Ghana': ['-', 0.367, '-', '-'],
 'Guatemala': ['-', 0.374, '-', '-'],
 'Haiti': [0.201, '-', '-', '-'],
 'India': [0.137, 0.375, 0.423, 0.443],
 'Indonesia': [0.328, 0.398, 0.5, 0.609],
 'Iran': ['-', '-', 0.477, 0.405],
 'Italy': ['-', '-', '-', 0.43],
 'Jordan': ['-', 0.464, '-', 0.436],
 'Kazakhstan': ['-', '-', 0.604, '-'],
 'Kenya'

In [102]:
all_topic_all_country_perincomedf

,Unnamed: 0,Income level,Forget ratio,Total images per income level,Forgotten images,dominant language,Recognized images,Recall
0,0,poor,0.776149,11119,8630,English,2489,0.223851
1,1,lower-middle,0.640902,10997,7048,English,3949,0.359098
2,2,upper-middle,0.543945,11446,6226,English,5220,0.456055
3,3,rich,0.564045,12054,6799,English,5255,0.435955
